<a href="https://colab.research.google.com/github/RKrumova/-/blob/main/2001321060.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#imports
import pandas as pd
import numpy as np
import sklearn
import ssl
from sklearn.linear_model import LogisticRegression
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt: ignored

   - Perform necessary preprocessing steps, such as removing irrelevant features/columns.
   - Handle any missing values or inconsistencies in the data.
   - Convert categorical features into numerical representations using techniques like one-hot encoding or label encoding.

In [17]:
train_data = pd.read_csv('train_data.csv')
train_data = pd.read_csv('test_data.csv')

train_data.isnull().sum()
#
train_data.dropna(inplace=True)
# turning Altitude from string to numeric
train_data['Altitude'] = pd.to_numeric(train_data['Altitude'], errors='coerce')
train_data['Altitude'].fillna(train_data['Altitude'].mean(), inplace=True)
# fill altitude
train_data['Altitude'].fillna(train_data['Altitude'].mean(), inplace=True)

# fix for harvest
train_data['Harvest.Year.Start'] = train_data['Harvest.Year'].str.split(' / ').str[0]
train_data['Harvest.Year.End'] = train_data['Harvest.Year'].str.split(' / ').str[1]
train_data['Harvest.Year.Start'] = pd.to_numeric(train_data['Harvest.Year.Start'], errors='coerce')
train_data['Harvest.Year.End'] = pd.to_numeric(train_data['Harvest.Year.End'], errors='coerce')
# fill the missing data
train_data['Harvest.Year.Start'].fillna(train_data['Harvest.Year.Start'].median(), inplace=True)
train_data['Harvest.Year.End'].fillna(train_data['Harvest.Year.End'].median(), inplace=True)
train_data['Farm.Name'].fillna('Unknown', inplace=True)
train_data['Mill'].fillna('Unknown', inplace=True)
train_data['Owner'].fillna(train_data['Owner'].mode()[0], inplace=True)
train_data['Processing.Method'].fillna(train_data['Processing.Method'].mode()[0], inplace=True)
train_data.isnull().sum()



Id                       0
Species                  0
Owner                    0
Country.of.Origin        0
Farm.Name                0
Lot.Number               0
Mill                     0
ICO.Number               0
Company                  0
Altitude                 0
Region                   0
Producer                 0
Number.of.Bags           0
Bag.Weight               0
In.Country.Partner       0
Harvest.Year             0
Grading.Date             0
Owner.1                  0
Variety                  0
Processing.Method        0
Moisture                 0
Category.One.Defects     0
Quakers                  0
Color                    0
Category.Two.Defects     0
Expiration               0
Certification.Body       0
Certification.Address    0
Certification.Contact    0
unit_of_measurement      0
altitude_low_meters      0
altitude_high_meters     0
altitude_mean_meters     0
Harvest.Year.Start       0
Harvest.Year.End         0
dtype: int64

In [ ]:
# train_data = pd.read_csv('train_data.csv')
X = train_data.drop(columns=['Coffee.Quality'])
y = train_data['Coffee.Quality']
